<h1><center>Programming Exercise 7:<br>K-means Clustering and Principal Component Analysis</center></h1>

# 1. K-means Clustering

## 1.1 Implementing K-means

## 1.2 K-means on Example Dataset

## 1.3 Random Initialization

## 1.4 Image Compression with K-means

## 1.5 Optional Exercise: Use Your Image

# 2. Principal Component Analysis

## 2.1 Example Dataset

## 2.2 Implementing PCA

## 2.3 Dimensionality Reduction with PCA

## 2.4 Face Image Dataset

## 2.5 Optional Exercise: PCA for Visualization 